In [48]:
from reload_recursive import reload_recursive
import mri_data
import monai_training

import os
import re
import shutil
import json
from pathlib import Path
from loguru import logger
import platform
from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

In [29]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_lesjak, DataSet, filter_first_ses  # noqa: E402, F401
from monai_training import preprocess  # noqa: E402

In [16]:
dataroot = Path("/media/srs-9/Data/lesjak_2017/data")
#! Set these variables
work_dir_name = "choroid_pineal_pituitary2"
train_dataset_file_name = "training-dataset.json"
prediction_postfix = "choroid_pineal_pituitary2_pred"
task_name = "infer_choroid_pineal_pituitary"
modalities = ["flair", "t1"]

In [9]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name
save_dir = Path("/media/srs-9/Data/lesjak_2017_predictions")

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

flair.t1_choroid_pineal_pituitary2_pred.nii.gz


In [47]:
dataset_proc = preprocess.DataSetProcesser.new_dataset(dataroot, scan_lesjak)
dataset_proc.prepare_images(["FLAIR", "T1W"])

2024-11-18 15:21:39.203 | INFO     | monai_training.preprocess:prepare_images:107 - Prepare Images
  0%|          | 0/30 [00:00<?, ?it/s]2024-11-18 15:21:39.203 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /media/srs-9/Data/lesjak_2017/data/patient01/FLAIR.T1W.nii.gz /media/srs-9/Data/lesjak_2017/data/patient01/FLAIR.nii.gz /media/srs-9/Data/lesjak_2017/data/patient01/T1W.nii.gz
2024-11-18 15:21:43.732 | SUCCESS  | monai_training.preprocess:log:22 - Saved /media/srs-9/Data/lesjak_2017/data/patient01/FLAIR.T1W.nii.gz
  3%|▎         | 1/30 [00:04<02:11,  4.53s/it]2024-11-18 15:21:43.733 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /media/srs-9/Data/lesjak_2017/data/patient02/FLAIR.T1W.nii.gz /media/srs-9/Data/lesjak_2017/data/patient02/FLAIR.nii.gz /media/srs-9/Data/lesjak_2017/data/patient02/T1W.nii.gz
2024-11-18 15:21:48.429 | SUCCESS  | monai_training.preprocess:log:22 - Saved /media/srs-9/Data/lesjak_2017/data/patient02/FLAIR.T1W.nii.gz
  7%|▋         | 2/30

In [49]:
images = []
for scan in dataset_proc.dataset[:2]:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

logger.info(f"Will run inference on {len(images)} scans")

datalist = {"testing": images}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

task = {
    "name": task_name,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": str(work_dir / "datalist.json"),
    "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, taskfile_name)
with open(task_file, "w") as f:
    json.dump(task, f, indent=4)

2024-11-18 15:25:34.498 | INFO     | __main__:<module>:6 - Will run inference on 2 scans


In [50]:
input_cfg = task_file  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [52]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage",
    "output_dir": save_dir,
    "data_root_dir": dataroot,
    "output_postfix": prediction_postfix,
    "separate_folder": False,
}

pred = ensemble(pred_param={"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/2 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `we

2024-11-18 15:27:36,725 INFO image_writer.py:197 - writing: /media/srs-9/Data/lesjak_2017_predictions/patient01/FLAIR.T1W_choroid_pineal_pituitary2_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:  50%|█████     | 1/2 [00:55<00:55, 55.79s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is depreca

2024-11-18 15:28:37,577 INFO image_writer.py:197 - writing: /media/srs-9/Data/lesjak_2017_predictions/patient02/FLAIR.T1W_choroid_pineal_pituitary2_pred.nii.gz


Ensembling (rank 0)...: 100%|██████████| 2/2 [01:56<00:00, 58.34s/it]
